In [16]:
pip install biopython
pip install StringIO
# https://www.tutorialspoint.com/biopython/biopython_overview_of_blast.htm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
org_name = 'mko'
org_full_name = 'Methylomonas_koyamae'


In [ ]:
''' run in terminal
cd C:\Users\vickenlee\Documents\GitHub\BuildGEM\Methylomonas_koyamae_genome\refdatabase
makeblastdb -in GCF_002441955.1_ASM244195v1_cds_from_genomic.fna -parse_seqids -dbtype nucl -out refgeneDB
'''

In [1]:
org_name = 'mbry'
org_full_name = 'Methylocystis_bryophila'


In [ ]:
''' run in terminal
cd C:\Users\vickenlee\Documents\GitHub\BuildGEM\Methylocystis_bryophila_genome\refdatabase
makeblastdb -in GCF_002117405.1_ASM211740v1_cds_from_genomic.fna -parse_seqids -dbtype nucl -out refgeneDB
'''

In [ ]:
ATGGCGCGCATCCTCTTCTTCGTGTGGCCCCATTTTGCCTCTGTAGCAAGCGGTTTAAAACTCGCCCGGGATTTGCTTTC
TAGAGGACATGAGGTTTCCTACGTGGGGGTGGCGGACTGTAAGACATTCGTCGTCGCCCACGGTCACCAATTCATACCGC
TGTATGCGAGATGGCTACCGGAGACGCCGACCGATTCGGCGCATGAGACCGCAACCGACCAAAATTTCTGGGACCAACTC
AGAAGCGCGCGCGCAAGGGTGCGGACGGTTCGCGCCCTCCTCGACGCTTTGGTAGCCGATGAGGAAAGAGAGTTTCAGGA
GATTCTGAAGCTCAAGAAGCCGGACCTTTTGATAATTCACATCTCTGACTTCGAAGTAATAATACCATCCTTGTTGGCAT
ATGAGGCCGGAATCAAGTGCGCTTATCTTTATGACAACTTTGGCCGGAGCGAAGATTGCAGGGTGCCTCCAGTCCGGACT
GCTATTTTGCCGAATGGCGGCCGGGGAGCATCCCTGAGATTGTGGCTGGCTTGGAAAATTCTAAAAATTCGGCGCCGCGT
CGCCGGCGATGTCTACGGCTTGCTAGGACTTGCATATCTGCCCGCCCTTCGCGAGGCGAGCCGAAGAATCGCCGAACGAT
GTCGTTATCCGTGCGAGTTCGTGGACTCCACGGATTTAGCGCCCTTGAAGGCGCATCTCCCTGAAATTGTGCTTTGCCCA
CCCGAACTCGAATTTCCCCATCACGGGCGACCTGGTCGCTATTATTTCGGCTGCAACATCGACTTGCACAGAAACGAAGG
AGATTTTCCTTGGAGTAGCTTGGATACGCGGCCATTGGTCTATTGTAGCCTTGGGACAATGTCCTGGTTTAACAAAGACG
TCTACCAGCGATTTTTCCGAATTATAATTGCTGTTGCGGAACATAGACCTGAATTTCAGTGGGTCGTAAATGTTTATGAT
GTAGTGAATCCTGAGGAATTTGAGAATGTGCCCCAGAACGTCCATCTTGTAAAGCGAGCGCCCCAGCTCGCGATTTTGCA
AAGAAGCGCTGTCGCTATAGTTCACGGAGGCCCCAACACCGTCAAGGAATGCGCTTATTTCGGCGTGCCGATGATCGTTT
TTCCGCTAGGTCTGGATCAGCCCGGAACCGCAGCGAGGGTTGTGTATCACGGTCTCGGGGTCAGCGGAGACTTTTGTAGT
GTTACGAAGGAGCAGATGGATGCGTTGGTGGACAGAGTCGATCGAGACTCCTTCGTCCGAGTTCAGACGCGGCTGATAGG
AGAAACATTGCGGGCAGCCGATCTTGAACGATCCCCAGTTGATTTTGTTGAAAAGCTTCTTGACCCCGAAAGTGAAAATT
CTGGAAGGTGA


DSM21852_01220

In [2]:
import os
faa_paths=[]
for root, dirs, files in os.walk(org_full_name+'_genome'):
    if 'cds_from_genomic.fna' in files:
        faa_paths.append(root+'\\cds_from_genomic.fna')
if len(faa_paths) > 1:
    faa_file = [x for x in faa_paths if 'GCF' in x][0]
else:
    faa_file = faa_paths[0]
print(faa_file)

Methylocystis_bryophila_genome\ncbi_dataset\data\GCA_027925445.1\cds_from_genomic.fna


In [3]:
from Bio.Blast.Applications import NcbiblastnCommandline
database_file = org_full_name+'_genome\\refdatabase\\refgeneDB'
blastn_cline = NcbiblastnCommandline(query = faa_file, db = database_file, outfmt = 5, out = org_full_name+"_genome\\GenomeBlastToRefResults.xml") 
stdout, stderr = blastn_cline()

In [9]:
import pandas as pd
from Bio.Blast import NCBIXML
import re

df_gene_tag = pd.DataFrame(columns=["NEWgene", "NEWtag", "refgene", "reftag"])

E_VALUE_THRESH = 1e-20 
for record in NCBIXML.parse(open(org_full_name+"_genome\\GenomeBlastToRefResults.xml")): 
   if record.alignments: 
      NEWgene = record.query.split(' ')[0]
      match = re.search(r'locus_tag=([\w\d_]+)', record.query)
      if match:
          NEWtag = match.group(1)
      else:
         NEWtag = ''

      for align in record.alignments: 
         for hsp in align.hsps: 
            if hsp.expect < E_VALUE_THRESH: 
               refgene = align.title.split(' ')[0]
               match = re.search(r'locus_tag=([\w\d_]+)', align.title)
               if match:
                  reftag = match.group(1)
               else:
                  reftag = ''
               df_gene_tag.loc[len(df_gene_tag.index)]=[NEWgene,NEWtag,refgene,reftag]
gene_tag = df_gene_tag.drop_duplicates(subset=['NEWgene','NEWtag','refgene','reftag'], keep='first')

In [5]:
record.query

'lcl|AP027150.1_cds_BDV41077.1_4330 [locus_tag=DSM21852_43310] [protein=hypothetical protein] [protein_id=BDV41077.1] [location=173707..174078] [gbkey=CDS]'

In [10]:
from Bio import SeqIO
import pandas as pd


df_gene_old_tag = pd.DataFrame(columns=["reftag", "oldtag"])

gbff_file = org_full_name+'_genome\\refdatabase\\GCF_002441955.1_ASM244195v1_genomic.gbff'  # Methylomonas_koyamae
gbff_file = org_full_name+'_genome\\refdatabase\\GCF_002117405.1_ASM211740v1_genomic.gbff'  # Methylocystis_bryophila

for record in SeqIO.parse(gbff_file, "genbank"):
    for feature in record.features:
        if feature.type == "CDS":
            locus_tag = feature.qualifiers.get("locus_tag", [""])[0]
            old_locus_tag = feature.qualifiers.get("old_locus_tag", [""])[0]
            if locus_tag != '':
                df_gene_old_tag.loc[len(df_gene_old_tag.index)]=[locus_tag,old_locus_tag]


In [11]:
import pandas as pd

df_tag = pd.merge(gene_tag, df_gene_old_tag, how='left', on='reftag')

In [12]:
df_tag.to_csv(org_full_name+'_genome\\'+org_name+'_gene_tag.csv', index=False,sep='\t')